In [5]:
import torch
import torch.nn as nn
from Models import TemporalDecoder, TemporalEncoder, DataDiffuser, TransitionNet, SimpleImageDecoder, SimpleImageEncoder, PositionalEncoder, StupidPositionalEncoder
import torch.optim as optim
from torchvision.utils import save_image
import torchvision
from torchvision import datasets, transforms
import numpy as np


def add_noise(x):
    """
    [0, 1] -> [0, 255] -> add noise -> [0, 1]
    """
    noise = x.new().resize_as_(x).uniform_()
    x = x * 255 + noise
    x = x / 256
    return x

def getMNISTDataLoader(bs):
    # MNIST Dataset
    train_dataset = datasets.MNIST(root='./mnist_data/', train=True, download=True, transform=transforms.Compose([
                                      transforms.Resize((32, 32)),
                                      #transforms.ToTensor(),
                                      #add_noise,
                                      ToTensor(),
        AddUniformNoise()
                                  ]))
    test_dataset = datasets.MNIST(root='./mnist_data/', train=False, download=True,
                                  transform=transforms.Compose([
                                      transforms.Resize((32, 32)),
                                      #transforms.ToTensor(),
                                      #add_noise,
                                      ToTensor(),
                                      AddUniformNoise()
                                  ]))

    # Data Loader (Input Pipeline)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

    return train_loader, test_loader

def getCIFAR10DataLoader(bs):
    # MNIST Dataset
    train_dataset = datasets.CIFAR10(root='./cifar10_data/', train=True, download=True, transform=transforms.Compose([
                                      transforms.Resize(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      add_noise,
                                      # transforms.ToTensor()
                                  ]))
    test_dataset = datasets.CIFAR10(root='./cifar10_data/', train=False, download=True,
                                  transform=transforms.Compose([
                                      transforms.Resize(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      add_noise,
                                      # transforms.ToTensor()
                                  ]))

    # Data Loader (Input Pipeline)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

    return train_loader, test_loader


def logit(x, alpha=1E-6):
    y = alpha + (1.-2*alpha)*x
    return np.log(y) - np.log(1. - y)


def logit_back(x, alpha=1E-6):
    y = torch.sigmoid(x)
    return (y - alpha)/(1.-2*alpha)


class AddUniformNoise(object):
    def __init__(self, alpha=1E-6):
        self.alpha = alpha
    def __call__(self,samples):
        samples = np.array(samples,dtype = np.float32)
        samples += np.random.uniform(size = samples.shape)
        samples = logit(samples/256., self.alpha)
        return samples


class ToTensor(object):
    def __init__(self):
        pass
    def __call__(self,samples):
        samples = torch.from_numpy(np.array(samples,dtype = np.float32)).float()
        return samples


class CNNDiffusionModel(nn.Module):
    def __init__(self, **kwargs):
        super(CNNDiffusionModel, self).__init__()
        self.T_MAX = kwargs['T_MAX']
        self.latent_s = kwargs['latent_s']
        self.t_emb_s = kwargs['t_emb_s']
        self.CNN = kwargs['CNN']
        self.register_buffer("beta_min", torch.tensor(kwargs['beta_min']))
        self.register_buffer("beta_max", torch.tensor(kwargs['beta_max']))
        self.device = 'cpu'
        self.img_size = [1, 32, 32]
        self.pos_enc = PositionalEncoder(self.t_emb_s // 2)  # StupidPositionalEncoder(T_MAX)  #
        self.simplified_trans = kwargs['simplified_trans']

        enc_net = [kwargs['enc_w']] * kwargs['enc_l']
        dec_net = [kwargs['dec_w']] * kwargs['dec_l']
        trans_net = [kwargs['trans_w']] * kwargs['trans_l']

        if self.CNN:
            self.enc = SimpleImageEncoder(self.img_size, self.latent_s, enc_net, t_dim=self.t_emb_s).to(dev)
            self.dec = SimpleImageDecoder(self.enc.features_dim, self.latent_s, dec_net, t_dim=self.t_emb_s,
                                          out_c=self.img_size[0]).to(dev)
        else:
            self.dec = TemporalDecoder(32**2, self.latent_s, dec_net, self.t_emb_s).to(dev)
            self.enc = TemporalEncoder(32**2, self.latent_s, enc_net, self.t_emb_s).to(dev)

        self.trans = TransitionNet(self.latent_s, trans_net, self.t_emb_s).to(dev)
        self.dif = DataDiffuser(beta_min=self.beta_min, beta_max=self.beta_max, t_max=self.T_MAX).to(dev)
        self.sampling_t0 = False

    def loss(self, x0):
        if self.sampling_t0:
            t0 = torch.randint(0, self.T_MAX - 1, [x0.shape[0]]).to(dev)
            x_t0, sigma_x_t0 = self.dif.diffuse(x0, t0, torch.zeros(x0.shape[0]).long().to(dev))
        else:
            t0 = torch.zeros(x0.shape[0]).to(dev).long()
            x_t0 = x0

        z_t0 = self.enc(x_t0.view(-1, *self.img_size), self.pos_enc(t0.float().unsqueeze(1)))
        # z_t0 = z_t0 + torch.randn(z_t0.shape).to(dev) * (1 - dif.alphas[t0]).sqrt().unsqueeze(1).expand(-1, z_t0.shape[1])
        t = torch.torch.distributions.Uniform(t0.float() + 1, torch.ones_like(t0) * self.T_MAX).sample().long().to(dev)

        z_t, sigma_z = self.dif.diffuse(z_t0, t, t0)
        x_t, sigma_x = self.dif.diffuse(x_t0, t, t0)

        mu_x_pred = self.dec(z_t, self.pos_enc(t.float().unsqueeze(1)))
        KL_x = ((mu_x_pred - x_t.view(bs, *self.img_size)) ** 2).view(bs, -1).sum(1) / sigma_x ** 2

        if self.simplified_trans:
            alpha_bar_t = self.dif.alphas[t].unsqueeze(1)#.expand(-1, self.latent_s)
            alpha_t = self.dif.alphas_t[t].unsqueeze(1)#.expand(-1, self.latent_s)
            beta_t = self.dif.betas[t].unsqueeze(1)#.expand(-1, self.latent_s)

            mu_z_pred = (z_t - beta_t/(1-alpha_bar_t).sqrt() * self.trans(z_t, self.pos_enc(t.float().unsqueeze(1))))/alpha_t.sqrt()
        else:
            mu_z_pred = self.trans(z_t, self.pos_enc(t.float().unsqueeze(1)))
        mu, sigma = self.dif.prev_mean(z_t0, z_t, t)

        KL_z = ((mu - mu_z_pred) ** 2).sum(1) / sigma ** 2

        loss = KL_x.mean(0) + KL_z.mean(0)

        return loss

    def to(self, device):
        super().to(device)
        self.device = device
        return self

    def sample(self, nb_samples=1):
        zT = torch.randn(64, self.latent_s).to(self.device)
        z_t = zT
        for t in range(self.T_MAX - 1, 0, -1):
            t_t = torch.ones(64, 1).to(self.device) * t
            if t > 0:
                sigma = ((1 - self.dif.alphas[t - 1]) / (1 - self.dif.alphas[t]) * self.dif.betas[t]).sqrt()
            else:
                sigma = 0
            if self.simplified_trans:
                alpha_bar_t = self.dif.alphas[t]
                alpha_t = self.dif.alphas_t[t]
                beta_t = self.dif.betas[t]
                mu_z_pred = (z_t - beta_t / (1 - alpha_bar_t).sqrt() * self.trans(z_t, self.pos_enc(t_t))) / alpha_t.sqrt()
            else:
                mu_z_pred = self.trans(z_t, self.pos_enc(t_t))
            z_t = mu_z_pred + torch.randn(z_t.shape, device=self.device) * sigma

        x_0 = self.dec(z_t, self.pos_enc(torch.zeros((nb_samples, 1), device=self.device))).view(nb_samples, -1)

        return x_0


import wandb
wandb.init(project="latent_diffusion", entity="awehenkel")


if __name__ == "__main__":
    bs = 100
    config = {
        'data': 'MNIST',
        'T_MAX': 20,
        'latent_s': 60,
        't_emb_s': 30,
        'CNN': False,
        'enc_w': 300,
        'enc_l': 4,
        'dec_w': 200,
        'dec_l': 3,
        'trans_w': 200,
        'trans_l': 3,
        "beta_min": 1e-2,
        "beta_max": .95,
        'simplified_trans': True
    }
    wandb.config.update(config)
    train_loader, test_loader = getMNISTDataLoader(bs)
    img_size = [1, 32, 32]

    # Compute Mean abd std per pixel
    x_mean = 0
    x_mean2 = 0
    for batch_idx, (cur_x, target) in enumerate(train_loader):
        cur_x = cur_x.view(bs, -1).float()
        x_mean += cur_x.mean(0)
        x_mean2 += (cur_x ** 2).mean(0)
    x_mean /= batch_idx + 1
    x_std = (x_mean2 / (batch_idx + 1) - x_mean ** 2) ** .5
    x_std[x_std == 0.] = 1.

    dev = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = CNNDiffusionModel(**config).to(dev)

    optimizer = optim.Adam(model.parameters(), lr=.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

    wandb.watch(model)
    def get_X_back(x):
        nb_x = x.shape[0]
        x = x * x_std.to(dev).unsqueeze(0).expand(nb_x, -1) + x_mean.to(dev).unsqueeze(0).expand(nb_x, -1)
        return logit_back(x)


    #sample = list(train_loader)[0][0][[0]].expand(bs, -1, -1, -1)
    #save_image(get_X_back(sample.to(dev)[[0]].reshape(1, -1)).reshape(1, 3, 32, 32), './Samples/Generated/sample_rel_' + '.png')
    def train(epoch):
        train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            #data = sample
            x0 = data.view(data.shape[0], -1).to(dev)

            x0 = (x0 - x_mean.to(dev).unsqueeze(0).expand(bs, -1)) / x_std.to(dev).unsqueeze(0).expand(bs, -1)
            optimizer.zero_grad()

            loss = model.loss(x0)

            loss.backward()
            train_loss += loss.item()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item() / len(data)))
        samples = get_X_back(model.sample(64)).view(64, *img_size)
        save_image(samples, './Samples/Generated/sample_gen_' + str(epoch) + '.png')
        scheduler.step(train_loss)
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
        wandb.log({"Train Loss": train_loss / len(train_loader.dataset), "Samples": [wandb.Image(samples)]})




Train Loss,19.00525
_step,44
_runtime,1324
_timestamp,1612877396


Train Loss,█▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██


wandb: wandb version 0.10.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
for i in range(150):
    train(i)

Train Epoch: 0 [0/60000 (0%)]	Loss: 31.115667
Train Epoch: 0 [10000/60000 (17%)]	Loss: 20.980425
Train Epoch: 0 [20000/60000 (33%)]	Loss: 19.927201
Train Epoch: 0 [30000/60000 (50%)]	Loss: 28.900093
Train Epoch: 0 [40000/60000 (67%)]	Loss: 25.488660
Train Epoch: 0 [50000/60000 (83%)]	Loss: 21.631567
====> Epoch: 0 Average loss: 22.5364
Train Epoch: 1 [0/60000 (0%)]	Loss: 23.405984
Train Epoch: 1 [10000/60000 (17%)]	Loss: 18.876420
Train Epoch: 1 [20000/60000 (33%)]	Loss: 18.553192
Train Epoch: 1 [30000/60000 (50%)]	Loss: 24.401355
Train Epoch: 1 [40000/60000 (67%)]	Loss: 21.924634
Train Epoch: 1 [50000/60000 (83%)]	Loss: 17.569568
====> Epoch: 1 Average loss: 20.9040
Train Epoch: 2 [0/60000 (0%)]	Loss: 28.151648
Train Epoch: 2 [10000/60000 (17%)]	Loss: 25.509075
Train Epoch: 2 [20000/60000 (33%)]	Loss: 20.306992
Train Epoch: 2 [30000/60000 (50%)]	Loss: 18.680603
Train Epoch: 2 [40000/60000 (67%)]	Loss: 18.621115
Train Epoch: 2 [50000/60000 (83%)]	Loss: 22.925486
====> Epoch: 2 Average 

Train Epoch: 24 [10000/60000 (17%)]	Loss: 17.331044
Train Epoch: 24 [20000/60000 (33%)]	Loss: 17.425044
Train Epoch: 24 [30000/60000 (50%)]	Loss: 16.777572
Train Epoch: 24 [40000/60000 (67%)]	Loss: 19.006439
Train Epoch: 24 [50000/60000 (83%)]	Loss: 17.049421
====> Epoch: 24 Average loss: 18.9973
Train Epoch: 25 [0/60000 (0%)]	Loss: 17.985801
Train Epoch: 25 [10000/60000 (17%)]	Loss: 18.688601
Train Epoch: 25 [20000/60000 (33%)]	Loss: 20.762185
Train Epoch: 25 [30000/60000 (50%)]	Loss: 19.256757
Train Epoch: 25 [40000/60000 (67%)]	Loss: 16.668174
Train Epoch: 25 [50000/60000 (83%)]	Loss: 17.603662
====> Epoch: 25 Average loss: 18.7107
Train Epoch: 26 [0/60000 (0%)]	Loss: 19.779572
Train Epoch: 26 [10000/60000 (17%)]	Loss: 22.089622
Train Epoch: 26 [20000/60000 (33%)]	Loss: 18.952891
Train Epoch: 26 [30000/60000 (50%)]	Loss: 17.694463
Train Epoch: 26 [40000/60000 (67%)]	Loss: 19.193444
Train Epoch: 26 [50000/60000 (83%)]	Loss: 18.844316
====> Epoch: 26 Average loss: 18.8158
Train Epoch:

Train Epoch: 47 [50000/60000 (83%)]	Loss: 18.056073
====> Epoch: 47 Average loss: 18.4853
Train Epoch: 48 [0/60000 (0%)]	Loss: 21.891936
Train Epoch: 48 [10000/60000 (17%)]	Loss: 18.003358
Train Epoch: 48 [20000/60000 (33%)]	Loss: 16.921926
Train Epoch: 48 [30000/60000 (50%)]	Loss: 21.360952
Train Epoch: 48 [40000/60000 (67%)]	Loss: 22.238164
Train Epoch: 48 [50000/60000 (83%)]	Loss: 21.913506
====> Epoch: 48 Average loss: 18.4847
Train Epoch: 49 [0/60000 (0%)]	Loss: 14.740614
Train Epoch: 49 [10000/60000 (17%)]	Loss: 18.933683
Train Epoch: 49 [20000/60000 (33%)]	Loss: 19.191704
Train Epoch: 49 [30000/60000 (50%)]	Loss: 18.402529
Train Epoch: 49 [40000/60000 (67%)]	Loss: 16.897047
Train Epoch: 49 [50000/60000 (83%)]	Loss: 16.665353
====> Epoch: 49 Average loss: 18.5505
Train Epoch: 50 [0/60000 (0%)]	Loss: 18.633685
Train Epoch: 50 [10000/60000 (17%)]	Loss: 17.680530
Train Epoch: 50 [20000/60000 (33%)]	Loss: 15.143097
Train Epoch: 50 [30000/60000 (50%)]	Loss: 16.860419
Train Epoch: 50 [

Train Epoch: 71 [40000/60000 (67%)]	Loss: 19.706300
Train Epoch: 71 [50000/60000 (83%)]	Loss: 17.849021
====> Epoch: 71 Average loss: 18.3173
Train Epoch: 72 [0/60000 (0%)]	Loss: 16.067274
Train Epoch: 72 [10000/60000 (17%)]	Loss: 14.322343
Train Epoch: 72 [20000/60000 (33%)]	Loss: 20.539160
Train Epoch: 72 [30000/60000 (50%)]	Loss: 17.987850
Train Epoch: 72 [40000/60000 (67%)]	Loss: 18.568621
Train Epoch: 72 [50000/60000 (83%)]	Loss: 17.493661
====> Epoch: 72 Average loss: 18.2863
Train Epoch: 73 [0/60000 (0%)]	Loss: 18.451920
Train Epoch: 73 [10000/60000 (17%)]	Loss: 15.756941
Train Epoch: 73 [20000/60000 (33%)]	Loss: 17.208945
Train Epoch: 73 [30000/60000 (50%)]	Loss: 20.346716
Train Epoch: 73 [40000/60000 (67%)]	Loss: 20.052485
Train Epoch: 73 [50000/60000 (83%)]	Loss: 15.448701
====> Epoch: 73 Average loss: 18.3271
Train Epoch: 74 [0/60000 (0%)]	Loss: 20.874512
Train Epoch: 74 [10000/60000 (17%)]	Loss: 19.658965
Train Epoch: 74 [20000/60000 (33%)]	Loss: 19.871985
Train Epoch: 74 [

Train Epoch: 95 [20000/60000 (33%)]	Loss: 17.730042
Train Epoch: 95 [30000/60000 (50%)]	Loss: 17.813217
Train Epoch: 95 [40000/60000 (67%)]	Loss: 15.738013
Train Epoch: 95 [50000/60000 (83%)]	Loss: 20.709387
====> Epoch: 95 Average loss: 18.1814
Train Epoch: 96 [0/60000 (0%)]	Loss: 17.052660
Train Epoch: 96 [10000/60000 (17%)]	Loss: 19.269821
Train Epoch: 96 [20000/60000 (33%)]	Loss: 18.353818
Train Epoch: 96 [30000/60000 (50%)]	Loss: 20.630769
Train Epoch: 96 [40000/60000 (67%)]	Loss: 22.184312
Train Epoch: 96 [50000/60000 (83%)]	Loss: 18.247047
====> Epoch: 96 Average loss: 18.2038
Train Epoch: 97 [0/60000 (0%)]	Loss: 15.285146
Train Epoch: 97 [10000/60000 (17%)]	Loss: 17.706404
Train Epoch: 97 [20000/60000 (33%)]	Loss: 16.948573
Train Epoch: 97 [30000/60000 (50%)]	Loss: 19.307715
Train Epoch: 97 [40000/60000 (67%)]	Loss: 16.193920
Train Epoch: 97 [50000/60000 (83%)]	Loss: 16.682319
====> Epoch: 97 Average loss: 18.1677
Train Epoch: 98 [0/60000 (0%)]	Loss: 18.686443
Train Epoch: 98 [

Train Epoch: 118 [30000/60000 (50%)]	Loss: 18.174150
Train Epoch: 118 [40000/60000 (67%)]	Loss: 17.407310
Train Epoch: 118 [50000/60000 (83%)]	Loss: 18.139017
====> Epoch: 118 Average loss: 18.1465
Train Epoch: 119 [0/60000 (0%)]	Loss: 16.069698
Train Epoch: 119 [10000/60000 (17%)]	Loss: 18.437675
Train Epoch: 119 [20000/60000 (33%)]	Loss: 17.095336
Train Epoch: 119 [30000/60000 (50%)]	Loss: 19.071021
Train Epoch: 119 [40000/60000 (67%)]	Loss: 15.007584
Train Epoch: 119 [50000/60000 (83%)]	Loss: 15.246648
====> Epoch: 119 Average loss: 18.0683
Train Epoch: 120 [0/60000 (0%)]	Loss: 17.140422
Train Epoch: 120 [10000/60000 (17%)]	Loss: 16.315746
Train Epoch: 120 [20000/60000 (33%)]	Loss: 17.970836
Train Epoch: 120 [30000/60000 (50%)]	Loss: 16.482283
Train Epoch: 120 [40000/60000 (67%)]	Loss: 20.402031
Train Epoch: 120 [50000/60000 (83%)]	Loss: 21.443118
====> Epoch: 120 Average loss: 18.0523
Train Epoch: 121 [0/60000 (0%)]	Loss: 16.345558
Train Epoch: 121 [10000/60000 (17%)]	Loss: 21.2985

Train Epoch: 141 [30000/60000 (50%)]	Loss: 18.166931
Train Epoch: 141 [40000/60000 (67%)]	Loss: 15.557900
Train Epoch: 141 [50000/60000 (83%)]	Loss: 16.393881
====> Epoch: 141 Average loss: 18.1108
Train Epoch: 142 [0/60000 (0%)]	Loss: 17.112643
Train Epoch: 142 [10000/60000 (17%)]	Loss: 15.880242
Train Epoch: 142 [20000/60000 (33%)]	Loss: 16.861460
Train Epoch: 142 [30000/60000 (50%)]	Loss: 14.027518
Train Epoch: 142 [40000/60000 (67%)]	Loss: 18.037009
Train Epoch: 142 [50000/60000 (83%)]	Loss: 16.507836
====> Epoch: 142 Average loss: 18.1101
Train Epoch: 143 [0/60000 (0%)]	Loss: 16.202262
Train Epoch: 143 [10000/60000 (17%)]	Loss: 18.072213
Train Epoch: 143 [20000/60000 (33%)]	Loss: 20.728574
Train Epoch: 143 [30000/60000 (50%)]	Loss: 16.716987
Train Epoch: 143 [40000/60000 (67%)]	Loss: 17.794619
Train Epoch: 143 [50000/60000 (83%)]	Loss: 16.771727
====> Epoch: 143 Average loss: 18.0824
Train Epoch: 144 [0/60000 (0%)]	Loss: 17.911010
Train Epoch: 144 [10000/60000 (17%)]	Loss: 17.3920